# Classify and Summarize documents in your IDP workflow with generative AI using Amazon Bedrock
---
<div class="alert alert-block alert-info"> 
    <b>NOTE:</b>This notebook contains dependancies that are installed in a different notebook. Be sure that you have previously run the library install scripts at the top of 01-idp-genai-introduction and restart the kernel before executing this script.  
</div>

In this notebook, you will learn how to leverage Amazon Bedrock, a service that provides access to advanced foundation models, for intelligent document processing (IDP) tasks. IDP has become increasingly important for businesses as they seek to extract valuable insights from their documents and streamline document-centric processes.

You will explore two key IDP use cases: document classification and document summarization. Document classification involves categorizing documents based on their content, enabling efficient routing and processing. Document summarization, on the other hand, aims to generate concise summaries of lengthy documents, allowing for quick understanding of their contents.

Throughout the notebook, you will see how Amazon Bedrock's powerful multimodal capabilities, combined with Rhubarb, an open-source library, can simplify and accelerate IDP workflows. Rhubarb provides a user-friendly interface for interacting with Bedrock's models, abstracting away the complexities of prompt engineering and model invocation.

By the end of this notebook, you will have gained hands-on experience in leveraging Amazon Bedrock and Rhubarb for document classification and summarization tasks, paving the way for more efficient and intelligent document processing in your organization.



### Environment Setup
---
First, the necessary libraries and global variables are imported and initialized.

In [ ]:
#Import script libraries and create global variables
import json
import sagemaker
import boto3
from bedrockhelper import get_response_from_claude
from textractor.parsers import response_parser
from textractor import Textractor

s3 = boto3.client("s3")
session = boto3.Session()
role = sagemaker.get_execution_role()
data_bucket = sagemaker.Session().default_bucket()
region = sagemaker.Session().boto_region_name
extractor = Textractor(region_name=region)
print(f"SageMaker bucket is {data_bucket}, and SageMaker Execution Role is {role}. Current region is {region}")

## 1. Document Classification
---
To classify documents, the text is first extracted from the document and parsed into a format suitable for processing with a Foundation Models (FMs). This is done by uploading a multi-page PDF to Amazon S3 and using the Textractor library to call Amazon Textract, parse the results, and store them. Textractor utilizes the asynchronous processing capability of Textract for multi-page documents and waits for the processing to complete before returning the results.

Textractor provides a summary of the content for each page of the document.

In [ ]:


samples_file_name = 'samples/Sample1.pdf'

# first we upload the file to S3
s3.upload_file(Filename='../' + samples_file_name, Bucket=data_bucket, Key=samples_file_name)

document = extractor.start_document_text_detection(file_source="s3://" + data_bucket + "/" + samples_file_name,
    save_image=False)

print(document.pages)

### Categorize each page
---
The Claude FM from Anthropic can classify a document based on its content, given a list of classes. The code demonstrates how to format a prompt for Claude to classify each page of the document into one of the specified classes.

In [ ]:
def format_prompt(doc_text):
    return f"""

Given the document

<document>{doc_text}<document>

classify the document into the following classes

<classes>
DRIVERS_LICENSE
INSURANCE_ID
RECEIPT
BANK_STATEMENT
W2
MEETING_MINUTES
</classes>



return only the CLASS_NAME with no preamble or explanation. 
"""


for page in document.pages:
    prompt = format_prompt(page.get_text())
    response = get_response_from_claude(prompt)
    print(f"""Page {page.page_num} is class {response[0]}. There were {response[1]} input tokens and {response[2]} output tokens used.""")



### Classification using Multimodal capabilities of Amazon Bedrock
The Claude 3 models are multimodal, meaning they can accept both text and images as input. Rhubarb is an open-source library that makes it easy to build IDP solutions using the multimodal capabilities of Bedrock.

The code shows how to use Rhubarb's `ClassificationSysPrompt` system prompt for single-class classification and `MultiClassificationSysPrompt` system prompt for multi-class classification of document pages.

In [ ]:
from rhubarb import DocAnalysis, SystemPrompts, LanguageModels


da = DocAnalysis(file_path="../samples/Sample1.pdf", 
                 boto3_session=session,
                 modelId=LanguageModels.CLAUDE_HAIKU_V1,
                 system_prompt=SystemPrompts().ClassificationSysPrompt)
resp = da.run(message="""Given the document, classify the pages into the following classes
                        <classes>
                        DRIVERS_LICENSE  # a driver's license
                        INSURANCE_ID     # a medical insurance ID card
                        RECEIPT          # a store receipt
                        BANK_STATEMENT   # a bank statement
                        W2               # a W2 tax document
                        MOM              # a minutes of meeting or meeting notes
                        </classes>""")
resp

Or multi-class classification. Note that in Multi-class classification it is helpful to clarify the hierarchy of classes to the model in two different list of classes. This should typically match with your document taxonomy such as

```
FINANCIAL           (Level-2)
├── BANK_STATEMENT  (Level-1 leaf)
└── W2              (Level-1 leaf)

IDENTIFICATION      (Level-2)
├── DRIVERS_LICENSE (Level-1 leaf)
└── INSURANCE_ID    (Level-1 leaf)
```

And so on

In [ ]:
da = DocAnalysis(file_path="../samples/Sample1.pdf", 
                 boto3_session=session,
                 modelId=LanguageModels.CLAUDE_HAIKU_V1,
                 system_prompt=SystemPrompts().MultiClassificationSysPrompt)
resp = da.run(message="""Given the document, classify the pages into the following classes
                        <classes_level1>
                        DRIVERS_LICENSE  # a driver's license
                        INSURANCE_ID     # a medical insurance ID card
                        RECEIPT          # a store receipt
                        BANK_STATEMENT   # a bank statement
                        W2               # a W2 tax document
                        MOM              # a minutes of meeting or meeting notes
                        <classes_level1>
                        <classes_level2>
                        FINANCIAL        # a document related to finances of a person
                        IDENTIFICATION   # a personal document such as ID, membership cards, etc.
                        GENERAL          # any other general document
                        </classes_level2>""")
resp

## 2. Document Summarization
---

Bedrock FMs are well-suited for summarizing document contents into a concise and readable format. The code demonstrates how to upload a PDF document to Amazon S3, extract its text using Textractor, and prompt the FMs to summarize the document at different levels (page-level and whole document).


In [ ]:
# first we upload the file to S3
employee_file_name = 'samples/employee_enrollment.pdf'
s3.upload_file(Filename='../' + employee_file_name, Bucket=data_bucket, Key=employee_file_name)

document = extractor.start_document_text_detection(file_source="s3://" + data_bucket + "/" + employee_file_name,
    save_image=False)

### Perform page level summarization
---
We can loop through each page of results and ask for a page level summary. This will create a summary of the document per page which will be helpful if different pages contain discrete information that has to be summarized individually.

In [ ]:
def format_prompt(doc_text):
    return f"""

Given the document

<document>{doc_text}<document>

Give me a 50 word summary of this document that can be shown alongside search results. 

Return only the summary text with no preamble. 
"""

for page in document.pages:
    prompt = format_prompt(page.get_text())
    response = get_response_from_claude(prompt)
    print(f"""Page {page.page_num} Summary:\n{response[0]}\nThere were {response[1]} input tokens and {response[2]} output tokens used.""")

### Summarize the whole document
---
We can also pass the complete text in for a single summarization. This helps users summarize the entire document at once which can be useful for generating summaries of larger documents such as research papers

In [ ]:
def format_prompt(doc_text):
    return f"""

Given the document

<document>{doc_text}<document>

Give me a 50 word summary of this document that can be shown alongside search results. 

Return only the summary text with no preamble. 
"""

prompt = format_prompt(document.get_text())
response = get_response_from_claude(prompt)
print(f"""Summary:\n{response[0]}\nThere were {response[1]} input tokens and {response[2]} output tokens used.""")

### Summarization using Bedrock multi-modal capibilities
The Claude 3 family of models support both images and text as imput. 

Rhubarb can generate summaries of every page in the document.

In [ ]:
import boto3
session = boto3.Session()
from rhubarb import DocAnalysis

da = DocAnalysis(file_path="../samples/employee_enrollment.pdf", 
                 modelId=LanguageModels.CLAUDE_HAIKU_V1,
                 boto3_session=session)
resp = da.run(message="Give me a brief summary for each page.")
resp

### Perform full summarization
---
Or you can generate an overall summary of the entire document. In this case, we will override the default System Prompt which breaks down the response per page. Rhubarb comes with a Summary specific System Prompt for the model, available via `SystemPrompts`.

In [ ]:
from rhubarb import SystemPrompts

da = DocAnalysis(file_path="../samples/employee_enrollment.pdf", 
                 boto3_session=session,
                 modelId=LanguageModels.CLAUDE_HAIKU_V1,
                 system_prompt=SystemPrompts().SummarySysPrompt)
resp = da.run(message="Give me a brief summary of this document.")
resp

### Perform summarization of specific pages
---
You can also perform summarization of specific pages using the `pages` parameter.

In [ ]:
da = DocAnalysis(file_path="../samples/employee_enrollment.pdf", 
                 boto3_session=session,
                 modelId=LanguageModels.CLAUDE_HAIKU_V1,
                 system_prompt=SystemPrompts().SummarySysPrompt,
                 pages=[1,3])
resp = da.run(message="Give me a brief summary of this document.")
resp

### Streaming summaries
---
In some cases, you may want to stream the summaries for example let's say a real time chat application. You can easily do that using the `run_stream` method. Let's generate the full summary and stream it.

In [ ]:
da = DocAnalysis(file_path="../samples/employee_enrollment.pdf", 
                 boto3_session=session,
                 modelId=LanguageModels.CLAUDE_HAIKU_V1,
                 system_prompt=SystemPrompts().SummarySysPrompt)
for resp in da.run_stream(message="Give me a brief summary of this document."):
    if isinstance(resp, str):
        print(resp,end='')
    else:
        print("\n")
        print(resp)

## Cleanup
---
Finally, the code demonstrates how to clean up by deleting the sample files uploaded to Amazon S3 earlier in the notebook.

In [ ]:
s3.delete_object(Bucket=data_bucket, Key=samples_file_name)
s3.delete_object(Bucket=data_bucket, Key=employee_file_name)